# Détecteur de stéréotype sur la Syrie dans la presse française

In [1]:
import pandas as pd
df = pd.read_parquet("./dataset/gallica_presse_1.parquet")

In [5]:
# Informations sur le DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7499 entries, 0 to 7498
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   file_id          7499 non-null   object
 1   ocr              7499 non-null   object
 2   title            7499 non-null   object
 3   date             7499 non-null   object
 4   author           7499 non-null   object
 5   page_count       7499 non-null   int64 
 6   word_count       7499 non-null   int64 
 7   character_count  7499 non-null   int64 
 8   complete_text    7499 non-null   object
dtypes: int64(3), object(6)
memory usage: 527.4+ KB


## Nettoyage du Dataset

In [6]:
df["clean_text"] = df["complete_text"].str.lower()

In [7]:
import re

def clean_ocr(text):
    # retirer caractères OCR parasites
    text = re.sub(r"[■□█▌▐▮▯◼︎◻︎…•●]", " ", text)
    
    # retirer répétitions de ponctuation ou artefacts : --- ==== ___
    text = re.sub(r"[-=_]{2,}", " ", text)
    
    # espacement propre
    text = re.sub(r"\s+", " ", text)
    return text.strip()

df["clean_text"] = df["clean_text"].apply(clean_ocr)

In [8]:
def normalize_ocr_safe(text):
    subs = {
        "ﬁ": "fi",
        "ﬂ": "fl",
        "’": "'",
        "´": "'",
        "‘": "'",
        "“": '"',
        "”": '"'
    }
    for k, v in subs.items():
        text = text.replace(k, v)
    return text

df["clean_text"] = df["clean_text"].apply(normalize_ocr_safe)

In [9]:
junk_patterns = [
    r"abonnez[- ]?vous",
    r"publicité",
    r"télécharger l'application",
    r"retour à la une",
    r"mentions légales",
    r"copyright .*",
    r"tous droits réservés"
]

def remove_junk(text):
    for pat in junk_patterns:
        text = re.sub(pat, " ", text)
    return re.sub(r"\s+", " ", text).strip()

df["clean_text"] = df["clean_text"].apply(remove_junk)


## 1. Première détection de pertinence

In [20]:
stereotypes = {
    "assad_vs_daech": {
        "canonical_form": "Le conflit syrien se résume à un duel : soit Assad, soit Daech.",
        "description": (
            "Stéréotype central : Assad serait le seul rempart contre Daech. "
            "Cette vision binaire efface la pluralité des acteurs (opposition modérée, "
            "groupes locaux, factions civiles, Kurdes, islamistes non jihadistes). "
            "Elle est souvent mobilisée pour justifier le maintien du régime."
        ),
        "variants": [
            "C'est Assad ou Daech",
            "Assad est le seul rempart contre Daech",
            "Sans Assad, Daech prendra le pouvoir",
            "Assad ou les terroristes",
            "Assad ou le chaos",
            "Assad ou les islamistes",
            "Il n'y a pas d'alternative à Assad",
            "Entre Assad et Daech il n'y a rien",
            "Le régime protège la Syrie du terrorisme"
        ],
        "keywords": [
            "Assad",
            "Daech",
            "État islamique",
            "terroristes",
            "rempart",
            "moindre mal",
            "pas d'alternative",
            "chaos",
            "stabilité",
            "extrémistes"
        ]
    },

    "opposition_islamiste": {
        "canonical_form": "L'opposition syrienne est fondamentalement islamiste ou jihadiste.",
        "description": (
            "Idée reçue : l'opposition est décrite comme un bloc islamiste, effaçant sa diversité "
            "(civile, laïque, minoritaire, armée non jihadiste). "
            "Cette vision s'appuie souvent sur des références au Friday protests, au takbîr, "
            "ou à l'influence des Frères musulmans."
        ),
        "variants": [
            "L'opposition est islamiste",
            "Les rebelles sont des islamistes",
            "Les opposants sont tous extrémistes",
            "Les rebelles syriens sont des jihadistes",
            "L'opposition est dominée par les islamistes"
        ],
        "keywords": [
            "opposition",
            "rebelles",
            "islamistes",
            "extrémistes",
            "Frères musulmans",
            "takbir",
            "mosquée"
        ]
    },

    "opposition_absente": {
        "canonical_form": "Il n'existe pas d'opposition syrienne crédible ou significative.",
        "description": (
            "Idée reçue : la révolution syrienne n'a pas produit d'opposition structurée. "
            "Cela nie l'existence des comités locaux, de l'Armée syrienne libre, "
            "des institutions civiles locales et des cadres politiques du CNS."
        ),
        "variants": [
            "Il n'y a pas d'opposition crédible",
            "Les rebelles sont désorganisés",
            "Il n'y a pas d'alternative politique",
            "L'opposition syrienne est inexistante",
            "Personne pour remplacer Assad"
        ],
        "keywords": [
            "opposition",
            "rebelles",
            "ASL",
            "alternatives",
            "absence d'opposition",
            "CNS",
            "pas crédible"
        ]
    },

    "revolution_confessionnelle": {
        "canonical_form": "La révolution syrienne est un soulèvement sunnite confessionnel.",
        "description": (
            "Stéréotype : le conflit syrien serait avant tout confessionnel ou communautaire. "
            "L'autrice montre que les revendications initiales étaient politiques, "
            "et que des figures minoritaires participèrent au soulèvement."
        ),
        "variants": [
            "Les sunnites veulent renverser les minorités",
            "La révolution est confessionnelle",
            "Conflit sunnites contre alaouites",
            "Soulèvement sectaire"
        ],
        "keywords": [
            "sunnites",
            "alaouites",
            "communautaire",
            "confessionnel",
            "minorités",
            "sectaire"
        ]
    },

    "daech_fils_de_la_revolution": {
        "canonical_form": "Daech est issu de la révolution syrienne ou en est une composante.",
        "description": (
            "Idée fausse : Daech serait né des manifestations ou de l'ASL. "
            "L'autrice montre que Daech est principalement le produit de l'Irak post-2003 "
            "et poursuit un projet jihadiste distinct de la révolution."
        ),
        "variants": [
            "Daech vient de l'opposition syrienne",
            "Daech est une composante de la révolution",
            "Les rebelles ont créé Daech",
            "Daech est né en Syrie en 2011"
        ],
        "keywords": [
            "Daech",
            "État islamique",
            "EIIL",
            "jihadistes",
            "opposition",
            "rebelles",
            "Al-Qaïda"
        ]
    },

    "regime_etat_fort": {
        "canonical_form": "Le régime Assad est un État fort qui assure la stabilité.",
        "description": (
            "Idée reçue : le régime serait un acteur stabilisateur face au chaos. "
            "L'autrice différencie État fort (institutionnel) et État féroce (répressif), "
            "et montre la fragmentation actuelle du régime."
        ),
        "variants": [
            "Le régime est stable",
            "Assad assure la stabilité",
            "Le régime protège la Syrie",
            "Assad est un État fort",
            "Le régime lutte efficacement contre les jihadistes"
        ],
        "keywords": [
            "stabilité",
            "État fort",
            "sécurité",
            "armée syrienne",
            "stabilise",
            "ordre",
            "protection"
        ]
    }
}


In [23]:
from tqdm.auto import tqdm
tqdm.pandas()

In [21]:
all_keywords = set()

for stereotype in stereotypes.values():
    for word in stereotype["keywords"]:
        all_keywords.add(word.lower())

In [24]:
import re

def count_keywords(text, keywords):
    count = 0
    for kw in keywords:
        # recherche de mots entiers \b...\b (très important)
        if re.search(r"\b" + re.escape(kw) + r"\b", text):
            count += 1
    return count

df["keyword_count"] = df["clean_text"].progress_apply(lambda t: count_keywords(t, all_keywords))


  0%|          | 0/7499 [00:00<?, ?it/s]

In [38]:
MIN_KEYWORDS = 8

filtered_lvl1 = df[df["keyword_count"] >= MIN_KEYWORDS]

In [39]:
filtered_lvl1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 107 entries, 22 to 7382
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   file_id          107 non-null    object
 1   ocr              107 non-null    object
 2   title            107 non-null    object
 3   date             107 non-null    object
 4   author           107 non-null    object
 5   page_count       107 non-null    int64 
 6   word_count       107 non-null    int64 
 7   character_count  107 non-null    int64 
 8   complete_text    107 non-null    object
 9   clean_text       107 non-null    object
 10  keyword_hit      107 non-null    bool  
 11  variant_hit      107 non-null    bool  
 12  keyword_count    107 non-null    int64 
dtypes: bool(2), int64(4), object(7)
memory usage: 10.2+ KB


In [14]:
variant_patterns = []

for s in stereotypes.values():
    for v in s["variants"]:
        variant_patterns.append(re.escape(v.lower()))

variant_regex = r"|".join(variant_patterns)

In [15]:
df["variant_hit"] = df["clean_text"].str.contains(variant_regex, case=False)
filtered_lvl2 = df[df["variant_hit"]]

In [17]:
filtered_lvl2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   file_id          0 non-null      object
 1   ocr              0 non-null      object
 2   title            0 non-null      object
 3   date             0 non-null      object
 4   author           0 non-null      object
 5   page_count       0 non-null      int64 
 6   word_count       0 non-null      int64 
 7   character_count  0 non-null      int64 
 8   complete_text    0 non-null      object
 9   clean_text       0 non-null      object
 10  keyword_hit      0 non-null      bool  
 11  variant_hit      0 non-null      bool  
dtypes: bool(2), int64(3), object(7)
memory usage: 0.0+ bytes


In [44]:
from sentence_transformers import SentenceTransformer, util


model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")

# encode articles
filtered_lvl1["embedding"] = filtered_lvl1["clean_text"].progress_apply(lambda x: model.encode(x, convert_to_tensor=True))

# encode stereos
canonical_embeddings = {
    key: model.encode(st["canonical_form"], convert_to_tensor=True)
    for key, st in stereotypes.items()
}

# compute similarity (max per article)
def max_similarity(emb):
    sims = []
    for stereo_emb in canonical_embeddings.values():
        sims.append(util.cos_sim(emb, stereo_emb).item())
    return max(sims)

filtered_lvl1["semantic_score"] = filtered_lvl1["embedding"].progress_apply(max_similarity)

filtered_lvl3 = filtered_lvl1[filtered_lvl1["semantic_score"] > 0.4]


  0%|          | 0/107 [00:00<?, ?it/s]

/tmp/ipykernel_102561/4197854608.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_lvl1["embedding"] = filtered_lvl1["clean_text"].progress_apply(lambda x: model.encode(x, convert_to_tensor=True))


  0%|          | 0/107 [00:00<?, ?it/s]

/tmp/ipykernel_102561/4197854608.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_lvl1["semantic_score"] = filtered_lvl1["embedding"].progress_apply(max_similarity)


In [ ]:
# encode stereos
canonical_embeddings = {
    key: model.encode(st["canonical_form"], convert_to_tensor=True)
    for key, st in stereotypes.items()
}

# compute similarity (max per article)
def max_similarity(emb):
    sims = []
    for stereo_emb in canonical_embeddings.values():
        sims.append(util.cos_sim(emb, stereo_emb).item())
    return max(sims)

filtered_lvl1["semantic_score"] = filtered_lvl1["embedding"].progress_apply(max_similarity)

filtered_lvl3 = filtered_lvl1[filtered_lvl1["semantic_score"] > 0.4]

In [47]:
filtered_lvl3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7 entries, 3354 to 7382
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   file_id          7 non-null      object 
 1   ocr              7 non-null      object 
 2   title            7 non-null      object 
 3   date             7 non-null      object 
 4   author           7 non-null      object 
 5   page_count       7 non-null      int64  
 6   word_count       7 non-null      int64  
 7   character_count  7 non-null      int64  
 8   complete_text    7 non-null      object 
 9   clean_text       7 non-null      object 
 10  keyword_hit      7 non-null      bool   
 11  variant_hit      7 non-null      bool   
 12  keyword_count    7 non-null      int64  
 13  embedding        7 non-null      object 
 14  semantic_score   7 non-null      float64
dtypes: bool(2), float64(1), int64(4), object(8)
memory usage: 798.0+ bytes


## LLM

In [99]:
import requests
import json
import re
from typing import List, Dict, Any

OLLAMA_URL = "http://localhost:11434/api/generate"
MODEL_NAME = "mistral:7b"  # or "llama3:8b", "mistral", etc.

In [100]:
stereotype_id = "assad_vs_daech"
stereotype = stereotypes[stereotype_id]

In [101]:
def get_focus_chunk_for_stereotype(
    text: str,
    stereotype: Dict[str, Any],
    window_chars: int = 2000,
) -> str:
    """
    Extrait un seul chunk du texte, centré sur la zone où les mots-clés
    du stéréotype sont les plus denses.
    
    - text : texte complet du journal (clean_text)
    - stereotype : entrée du dict stereotypes[stereotype_id]
    - window_chars : taille maximale du chunk (en caractères)
    """

    text = text.strip()
    if not text:
        return ""

    keywords = [kw.lower() for kw in stereotype["keywords"]]
    # On construit une regex qui cherche n'importe quel mot-clé comme mot entier
    pattern = r"\b(" + "|".join(re.escape(kw) for kw in keywords) + r")\b"

    matches = list(re.finditer(pattern, text.lower(), flags=re.IGNORECASE))

    # S'il n'y a aucun mot-clé, on peut soit renvoyer le début du texte,
    # soit une chaîne vide. Ici, on renvoie le début du texte tronqué.
    if not matches:
        return text[:window_chars]

    # On récupère les positions (en caractères) de chaque match
    positions = [m.start() for m in matches]
    positions.sort()

    # On cherche la "fenêtre" de taille window_chars contenant le plus de mots-clés
    best_count = 1
    best_start_pos = positions[0]
    best_end_pos = positions[0]

    n = len(positions)
    j = 0
    for i in range(n):
        # on avance j tant que la distance reste <= window_chars
        while j + 1 < n and positions[j + 1] - positions[i] <= window_chars:
            j += 1
        count = j - i + 1
        if count > best_count:
            best_count = count
            best_start_pos = positions[i]
            best_end_pos = positions[j]

    # On centre le chunk sur le milieu de cette meilleure fenêtre
    center = (best_start_pos + best_end_pos) // 2
    half = window_chars // 2

    chunk_start = max(0, center - half)
    chunk_end = min(len(text), chunk_start + window_chars)

    return text[chunk_start:chunk_end]


In [102]:
def build_prompt_for_chunk(text: str, stereotype_id: str, stereotype: Dict[str, Any]) -> str:
    canonical = stereotype["canonical_form"]
    variants = stereotype["variants"]

    prompt = f"""
Tu es un assistant qui analyse des textes de journaux français.

Stéréotype à détecter (ID: {stereotype_id}) :
"{canonical}"

Variantes possibles de ce stéréotype :
- "{"\" \n- \"".join(variants)}"

Tâche :
On te donne un extrait de texte d'un journal français.
Tu dois dire si ce texte véhicule ce stéréotype, de manière explicite ou implicite.

Consignes :
- Si le texte reprend l'idée qu'il faut choisir entre Assad et Daech (ou Assad et les terroristes, les islamistes, le chaos), considère que le stéréotype est présent.
- S'il mentionne seulement Assad ou Daech sans opposer l'un à l'autre comme les "seules options", considère que le stéréotype est probablement absent ou incertain.
- Si le texte critique explicitement cette vision binaire, le stéréotype est présent mais dans un mode "critiqué".

Réponds STRICTEMENT au format JSON suivant (sans texte additionnel) :

{{
  "stereotype_id": "{stereotype_id}",
  "present": "yes" | "no" | "uncertain",
  "mode": "reproduit" | "nuancé" | "critiqué" | "non_pertinent",
  "score_presence": nombre entre 0 et 1,
  "score_intensite": nombre entre 0 et 1,
  "citations": ["liste de phrases EXACTES du texte qui justifient ta réponse"]
}}

Voici le texte à analyser :
---
{text}
---
"""
    return prompt


In [103]:
def analyze_chunk_with_ollama(
    text: str,
    stereotype_id: str,
    stereotype: Dict[str, Any],
    model: str = MODEL_NAME,
    url: str = OLLAMA_URL,
    temperature: float = 0.1,
    timeout: int = 120
) -> Dict[str, Any]:
    prompt = build_prompt_for_chunk(text, stereotype_id, stereotype)

    payload = {
        "model": model,
        "prompt": prompt,
        "format": "json",      # on demande du JSON directement
        "stream": False,
        "options": {
            "temperature": temperature
        }
    }

    resp = requests.post(url, json=payload, timeout=timeout)
    resp.raise_for_status()

    raw_response = resp.json().get("response", "").strip()

    try:
        data = json.loads(raw_response)
    except json.JSONDecodeError:
        data = {
            "error": "json_decode_error",
            "raw": raw_response
        }

    return data


In [104]:
def analyze_issue_for_stereotype_focus(
    full_text: str,
    stereotype_id: str,
    stereotype: Dict[str, Any],
    window_chars: int = 2000
) -> Dict[str, Any]:
    """
    Analyse un journal (texte complet) pour un stéréotype donné,
    en ne sélectionnant qu'un chunk "focal" autour des mots-clés.

    Retourne :
    - score_presence (ou 0.0 si pas de résultat)
    - mode
    - le chunk utilisé, pour inspection
    - le résultat brut du LLM
    """
    focus_chunk = get_focus_chunk_for_stereotype(
        text=full_text,
        stereotype=stereotype,
        window_chars=window_chars
    )

    if not focus_chunk.strip():
        return {
            "stereotype_id": stereotype_id,
            "score_presence": 0.0,
            "mode": "non_pertinent",
            "chunk": "",
            "llm_result": None
        }

    res = analyze_chunk_with_ollama(focus_chunk, stereotype_id, stereotype)

    if isinstance(res, dict) and "score_presence" in res and not res.get("error"):
        return {
            "stereotype_id": stereotype_id,
            "score_presence": res.get("score_presence", 0.0),
            "mode": res.get("mode", "non_pertinent"),
            "chunk": focus_chunk,
            "llm_result": res
        }
    else:
        return {
            "stereotype_id": stereotype_id,
            "score_presence": 0.0,
            "mode": "non_pertinent",
            "chunk": focus_chunk,
            "llm_result": res
        }


In [106]:
stereotype_id = "assad_vs_daech"
stereotype = stereotypes[stereotype_id]

def analyze_row_focus(row):
    text = row["clean_text"]
    return analyze_issue_for_stereotype_focus(
        full_text=text,
        stereotype_id=stereotype_id,
        stereotype=stereotype,
        window_chars=2000,  # tu peux ajuster (1500–3000)
    )

# Test d'abord sur un tout petit échantillon :
# df_sample = df_candidates.sample(5, random_state=0)
# df_sample["llm_assad_vs_daech"] = df_sample.apply(analyze_row_focus, axis=1)

filtered_lvl3["llm_assad_vs_daech"] = filtered_lvl3.progress_apply(analyze_row_focus, axis=1)


  0%|          | 0/7 [00:00<?, ?it/s]

/tmp/ipykernel_102561/1259570126.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_lvl3["llm_assad_vs_daech"] = filtered_lvl3.progress_apply(analyze_row_focus, axis=1)


In [107]:
filtered_lvl3["assad_daech_score"] = filtered_lvl3["llm_assad_vs_daech"].apply(
    lambda d: d.get("score_presence", 0.0) if isinstance(d, dict) else 0.0
)

filtered_lvl3["assad_daech_mode"] = filtered_lvl3["llm_assad_vs_daech"].apply(
    lambda d: d.get("mode", "non_pertinent") if isinstance(d, dict) else "non_pertinent"
)


/tmp/ipykernel_102561/619302398.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_lvl3["assad_daech_score"] = filtered_lvl3["llm_assad_vs_daech"].apply(
/tmp/ipykernel_102561/619302398.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_lvl3["assad_daech_mode"] = filtered_lvl3["llm_assad_vs_daech"].apply(


In [108]:
import json
from textwrap import indent, fill

def pretty_print_result(result: dict, max_chunk_chars: int = 500):
    """
    Nicely prints one LLM result as human-readable text.
    """
    if result is None or not isinstance(result, dict):
        print("❌ Invalid result")
        return

    stereotype = result.get("stereotype_id", "?")
    score = result.get("score_presence", "N/A")
    mode = result.get("mode", "N/A")
    chunk = result.get("chunk", "")
    llm_raw = result.get("llm_result", {})

    print("\n" + "="*80)
    print(f"📰  RESULT FOR STEREOTYPE: {stereotype}")
    print("="*80)
    print(f"⭐ Score presence:      {score}")
    print(f"🎭 Interpretation mode: {mode}")
    
    # Print LLM output (formatted JSON)
    print("\n📦 LLM RAW RESULT (JSON):")
    print(indent(json.dumps(llm_raw, indent=4, ensure_ascii=False), "  "))

    # Print the chunk used (truncated)
    print("\n🧩 TEXT CHUNK ANALYZED (truncated):")
    if len(chunk) > max_chunk_chars:
        chunk = chunk[:max_chunk_chars] + " [...]"
    print(indent(fill(chunk, width=100), "  "))
    
    print("="*80)


In [113]:
pretty_print_result(filtered_lvl3.iloc[4]["llm_assad_vs_daech"])



📰  RESULT FOR STEREOTYPE: assad_vs_daech
⭐ Score presence:      0.1
🎭 Interpretation mode: non_pertinent

📦 LLM RAW RESULT (JSON):
  {
      "stereotype_id": "assad_vs_daech",
      "present": "uncertain",
      "mode": "non_pertinent",
      "score_presence": 0.1,
      "score_intensite": 0.2,
      "citations": [
          "Il y a un singulier mirage dans la séduction exercée par la révolution russe sur nos socialistes et sur la confédération du travail elle-même."
      ]
  }

🧩 TEXT CHUNK ANALYZED (truncated):
  es, ou plutôt un seul sacrifice, le sacrifice des préjugés de classe car, s'ils le faisaient, ils
  retrouveraient bientôt, avec une quiétude d'esprit nouvelle, une récompense bien sonnante. mais
  c'est aussi aux salariés qu'il faut demander un sacrifice pareil. la troisième condition de la paix
  sociale que nous voulions signaler est précisément que les ouvriers devront renoncer à ce principe
  de la lutte des classes et apprendre à y voir ce qu'il enferme en réalité un

In [114]:
def pretty_print_top_results(df, column="llm_assad_vs_daech", top_n=5):
    """
    Pretty print the top N most relevant journals based on stereotype score.
    """
    # extract score
    df["_score"] = df[column].apply(
        lambda d: d.get("score_presence", 0.0) if isinstance(d, dict) else 0.0
    )

    top = df.sort_values("_score", ascending=False).head(top_n)

    print("\n" + "#"*80)
    print(f"📊 TOP {top_n} JOURNALS (BY SCORE)")
    print("#"*80)

    for idx, row in top.iterrows():
        print(f"\n\n=== 📝 JOURNAL INDEX {idx}  (score={row['_score']}) ===")
        pretty_print_result(row[column])
    
    # cleanup helper column
    df.drop(columns=["_score"], inplace=True, errors="ignore")


In [115]:
pretty_print_top_results(filtered_lvl3)



################################################################################
📊 TOP 5 JOURNALS (BY SCORE)
################################################################################


=== 📝 JOURNAL INDEX 3891  (score=0.2) ===

📰  RESULT FOR STEREOTYPE: assad_vs_daech
⭐ Score presence:      0.2
🎭 Interpretation mode: non_pertinent

📦 LLM RAW RESULT (JSON):
  {
      "stereotype_id": "assad_vs_daech",
      "present": "uncertain",
      "mode": "non_pertinent",
      "score_presence": 0.2,
      "score_intensite": 0.1,
      "citations": [
          "Les réformateurs sont divisés en deux partis les extrémistes et les modérés"
      ]
  }

🧩 TEXT CHUNK ANALYZED (truncated):
  lomate en vue a exprimé au sujet de l'édit l'opinion que voici. pour apprécier à leur juste valeur
  les déclarations impériales, il faut se rappeler que te chinois a des tendances à aller aux
  extrêmes, employer des termes d'une telle courtoisie qu'ils paraissent aux étrariyers d'une humilité
  obséquieuse

/tmp/ipykernel_102561/3202493500.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["_score"] = df[column].apply(
/tmp/ipykernel_102561/3202493500.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=["_score"], inplace=True, errors="ignore")


In [117]:
filtered_lvl3

,file_id,ocr,title,date,author,page_count,word_count,character_count,complete_text,clean_text,keyword_hit,variant_hit,keyword_count,embedding,semantic_score,chunks,llm_assad_vs_daech,assad_daech_score,assad_daech_mode
3354,bpt6k5679688,100,Le Matin : derniers télégrammes de la nuit,1906-05-15,None,22,34343,214927,\nComment __estji^ EXÉCUTE PAR LES RÉVOLUTIONN...,comment estji^ exécute par les révolutionnaire...,True,False,8,"[tensor(0.0386), tensor(0.0500), tensor(0.0246...",0.492723,[comment estji^ exécute par les révolutionnair...,"{'stereotype_id': 'assad_vs_daech', 'score_pre...",0.1,non_pertinent
3891,bpt6k5699649,100,Le Matin : derniers télégrammes de la nuit,1911-11-01,None,28,43088,266665,\n1 Ce qu'on voit à la lueur d'une catastrophe...,1 ce qu'on voit à la lueur d'une catastrophe l...,True,False,8,"[tensor(0.0201), tensor(0.0692), tensor(0.0547...",0.402799,[1 ce qu'on voit à la lueur d'une catastrophe ...,"{'stereotype_id': 'assad_vs_daech', 'score_pre...",0.2,non_pertinent
5573,bpt6k475335q,71,La Presse,1850-11-12,None,16,24844,153088,"\nPaîris, il iftvcmbre. \n•̃ ̃ .'̃̃̃""̃ \nP0MTH...","paîris, il iftvcmbre. ̃ ̃ .'̃̃̃""̃ p0mthh1 im i...",True,False,8,"[tensor(-0.0075), tensor(0.0591), tensor(0.032...",0.537428,"[paîris, il iftvcmbre. ̃ ̃ .'̃̃̃""̃ p0mthh1 im ...","{'stereotype_id': 'assad_vs_daech', 'score_pre...",0.2,non_pertinent
6957,bpt6k64534834,100,Cahiers Reconstruction : pour un socialisme dé...,1962,None,100,26317,171993,\ncahiers RECONSTRUCTION \nAlgérie Mouvement d...,cahiers reconstruction algérie mouvement des i...,True,False,8,"[tensor(0.0137), tensor(0.0665), tensor(0.0483...",0.450609,[cahiers reconstruction algérie mouvement des ...,"{'stereotype_id': 'assad_vs_daech', 'score_pre...",0.1,non_pertinent
7372,bpt6k14173m,96,Revue politique et parlementaire : questions p...,1920-01,None,1444,173383,1098623,\nRevue \nPolitique et Parlementaire \nTOME ÇI...,revue politique et parlementaire tome çii revu...,True,False,11,"[tensor(0.0189), tensor(0.0385), tensor(0.0537...",0.517571,[revue politique et parlementaire tome çii rev...,"{'stereotype_id': 'assad_vs_daech', 'score_pre...",0.1,non_pertinent
7373,bpt6k405160j,55,L'Humanité : journal socialiste quotidien,1934-10-17,Parti communiste français,22,34239,218367,\n(̃̃̃̃̃̃̃̃̃̃̃̃̃'̃̃̃̃̃̃̃̃̃̃̃̃̃̃̃̃̃̃̃̃̃̃«̃̃̃H n...,(̃̃̃̃̃̃̃̃̃̃̃̃̃'̃̃̃̃̃̃̃̃̃̃̃̃̃̃̃̃̃̃̃̃̃̃«̃̃̃h nii...,True,False,8,"[tensor(-0.0066), tensor(-0.0013), tensor(0.01...",0.480510,[(̃̃̃̃̃̃̃̃̃̃̃̃̃'̃̃̃̃̃̃̃̃̃̃̃̃̃̃̃̃̃̃̃̃̃̃«̃̃̃h ni...,"{'stereotype_id': 'assad_vs_daech', 'score_pre...",0.1,non_pertinent
7382,bpt6k4809130,80,Journal des débats politiques et littéraires,1902-08-17,None,16,29061,184189,\nNous rappelons à nos abonnés que toute deman...,nous rappelons à nos abonnés que toute demande...,True,False,8,"[tensor(0.0077), tensor(0.0260), tensor(0.0333...",0.497011,[nous rappelons à nos abonnés que toute demand...,"{'stereotype_id': 'assad_vs_daech', 'score_pre...",0.0,non_pertinent
